In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gff_file_path='./C_glabrata_CBS138_version_s02-m07-r06_features.csv'
CHIP_seq_file_path='./infection data merge_reliable peaks-for identify target genes.xlsx'
output_path='./infection data merge_reliable peaks_target genes_atg-V2.xlsx'

gff_header_rows=23
CHIP_seq_header_rows=0

genome_gff=pd.read_csv(gff_file_path,skiprows=gff_header_rows,names=['chr','Veu','Genome_attr','start','end','a','strand','b','ID'])
CHIP_seq=pd.read_excel(CHIP_seq_file_path,skiprows=CHIP_seq_header_rows)

In [3]:
genome_gff['Genome_attr'] = genome_gff['Genome_attr'].replace('pseudogene', 'gene')

In [4]:
chr_list=[]
chr_list.append(genome_gff.iloc[0][0])
for i in range(len(genome_gff)):
    flag=1
    for j in chr_list:
        if(genome_gff.iloc[i][0])==j:
            flag=0
            break
    if(flag==1): 
        chr_list.append(genome_gff.iloc[i][0])
    

In [5]:
CHIP_seq_output=CHIP_seq.copy()
#only need ['type', 'chr', 'start', 'end']
CHIP_seq_output = CHIP_seq_output.iloc[:,0:4]

In [6]:
CHIP_seq_output['abs_summit'] = (CHIP_seq_output['start'] + CHIP_seq_output['end'])/2
CHIP_seq_output['abs_summit'] = CHIP_seq_output['abs_summit'].astype(int)

In [7]:
CHIP_seq_output['ctrl_gene_down']=None
CHIP_seq_output['down_distance']=None

CHIP_seq_output['ctrl_gene_up']=None
CHIP_seq_output['up_distance']=None

CHIP_seq_output['ingenebody']=None
CHIP_seq_output['Tss_distance']=None

CHIP_seq_output['code_strand']=None

In [8]:
CHIP_seq_output

,type,chr,start,end,abs_summit,ctrl_gene_down,down_distance,ctrl_gene_up,up_distance,ingenebody,Tss_distance,code_strand
0,common,ChrA_C_glabrata_CBS138,57463,57663,57563,None,None,None,None,None,None,None
1,common,ChrA_C_glabrata_CBS138,130041,130241,130141,None,None,None,None,None,None,None
2,common,ChrA_C_glabrata_CBS138,160382,160582,160482,None,None,None,None,None,None,None
3,common,ChrA_C_glabrata_CBS138,186433,186633,186533,None,None,None,None,None,None,None
4,common,ChrA_C_glabrata_CBS138,194065,194265,194165,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
246,TH0_specific,ChrL_C_glabrata_CBS138,1094565,1094765,1094665,None,None,None,None,None,None,None
247,TH0_specific,ChrL_C_glabrata_CBS138,1350210,1350410,1350310,None,None,None,None,None,None,None
248,TH0_specific,ChrL_C_glabrata_CBS138,1425087,1425287,1425187,None,None,None,None,None,None,None
249,TH0_specific,ChrM_C_glabrata_CBS138,708771,708971,708871,None,None,None,None,None,None,None


In [9]:
CHIP_seq_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            251 non-null    object
 1   chr             251 non-null    object
 2   start           251 non-null    int64 
 3   end             251 non-null    int64 
 4   abs_summit      251 non-null    int32 
 5   ctrl_gene_down  0 non-null      object
 6   down_distance   0 non-null      object
 7   ctrl_gene_up    0 non-null      object
 8   up_distance     0 non-null      object
 9   ingenebody      0 non-null      object
 10  Tss_distance    0 non-null      object
 11  code_strand     0 non-null      object
dtypes: int32(1), int64(2), object(9)
memory usage: 22.7+ KB


In [10]:
#calculate the distance near TF bind site
for i in range(len(CHIP_seq_output)):
    for Chr_num in chr_list:
        if(CHIP_seq_output.iloc[i][1]==Chr_num):
            
            genome_gff_chr=genome_gff[genome_gff['chr']==Chr_num].copy()
            binding_site=CHIP_seq_output.iloc[i]['abs_summit']
            
            #TF bind DNA inside gene body?
            filter_1=(genome_gff_chr['start']<=binding_site)
            tem_genome_gff_chr=genome_gff_chr[filter_1].copy()
            filter_2=(tem_genome_gff_chr['end']>=binding_site)
            tem_genome_gff_chr=tem_genome_gff_chr[filter_2].copy()
            
            tem_genome_gff_chr=tem_genome_gff_chr[tem_genome_gff_chr['Genome_attr']=='gene'].copy()
            
            if(len(tem_genome_gff_chr)!=0):
                CHIP_seq_output.iloc[i,9]=tem_genome_gff_chr.iloc[0]['ID']
                if(tem_genome_gff_chr.iloc[0]['strand']=='+'):
                    CHIP_seq_output.iloc[i,10]=binding_site-tem_genome_gff_chr.iloc[0]['start']
                    CHIP_seq_output.iloc[i,11]='+'
                if(tem_genome_gff_chr.iloc[0]['strand']=='-'):
                    CHIP_seq_output.iloc[i,10]=tem_genome_gff_chr.iloc[0]['end']-binding_site
                    CHIP_seq_output.iloc[i,11]='-'
                    
            
            #leading strand
            filter_1=(genome_gff_chr['start']>=binding_site)
            tem_genome_gff_chr=genome_gff_chr[filter_1].copy()
            
            #gene
            tem_genome_gff_chr_gene = tem_genome_gff_chr[tem_genome_gff_chr['Genome_attr']=='gene'].copy()
            min_start=tem_genome_gff_chr_gene['start'].min()
            tem_genome_gff_chr_gene = tem_genome_gff_chr_gene[tem_genome_gff_chr_gene['start']==min_start].copy()
            
            if(len(tem_genome_gff_chr_gene)!=0):
                if(tem_genome_gff_chr_gene.iloc[0]['strand']=='+'):
                    
                    #CDS of gene
                    start =  tem_genome_gff_chr_gene.iloc[0]['start']
                    end =  tem_genome_gff_chr_gene.iloc[0]['end']
                    tem_genome_gff_chr_cds = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                    tem_genome_gff_chr_cds = tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['end']<=end].copy()
                    tem_genome_gff_chr_cds =  tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['Genome_attr']=='CDS'].copy()
                    min_start_cds = tem_genome_gff_chr_cds['start'].min()
                    tem_genome_gff_chr_cds = tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['start'] == min_start_cds].copy()
                    
                    # tRNA and rRNA without CDS, replace it 
                    if(len(tem_genome_gff_chr_cds)==0):
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['end']<=end].copy()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['Genome_attr']=='rRNA'].copy()
                        min_start_rRNA = tem_genome_gff_chr_rRNA['start'].min()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['start']==min_start_rRNA].copy()
                        
                        #not rRNA,tRNA?
                        if(len(tem_genome_gff_chr_rRNA)==0):
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['end']<=end].copy()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['Genome_attr']=='tRNA'].copy()
                            min_start_tRNA = tem_genome_gff_chr_tRNA['start'].min()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['start']==min_start_tRNA].copy()
                            
                            #not tRNA,rRNA and CDS, report error.
                            if(len(tem_genome_gff_chr_tRNA)==0):
                                print("error! There is no CDS & tRNA & rRNA in downstream!!!")
                                print(tem_genome_gff_chr_gene)
                                print("\n\n")
                            else:
                                CHIP_seq_output.iloc[i,5]=tem_genome_gff_chr_tRNA.iloc[0]['ID']
                                CHIP_seq_output.iloc[i,6]=tem_genome_gff_chr_tRNA.iloc[0]['start']-binding_site
                                
            
                        
                        else:
                            CHIP_seq_output.iloc[i,5]=tem_genome_gff_chr_rRNA.iloc[0]['ID']
                            CHIP_seq_output.iloc[i,6]=tem_genome_gff_chr_rRNA.iloc[0]['start']-binding_site
                        
                    else:
                        #write information to CHIP_seq_output
                        CHIP_seq_output.iloc[i,5]=tem_genome_gff_chr_cds.iloc[0]['ID']
                        CHIP_seq_output.iloc[i,6]=tem_genome_gff_chr_cds.iloc[0]['start']-binding_site
                
                else:
                    CHIP_seq_output.iloc[i,5]=None
                    CHIP_seq_output.iloc[i,6]=None
            else:
                print("Nearst downstream attribute is not gene!",end = " ")
                print("chr=%s, peak summit=%s"%(CHIP_seq_output.iloc[i][1], binding_site))
                print()

                
                            
                            
            #lagging strand
            filter_1=(genome_gff_chr['end']<=binding_site)
            tem_genome_gff_chr=genome_gff_chr[filter_1].copy()
            
            #find gene
            tem_genome_gff_chr_gene = tem_genome_gff_chr[tem_genome_gff_chr['Genome_attr']=='gene'].copy()
            max_end=tem_genome_gff_chr_gene['end'].max()
            tem_genome_gff_chr_gene = tem_genome_gff_chr_gene[tem_genome_gff_chr_gene['end']==max_end].copy()
            
            if(len(tem_genome_gff_chr_gene)!=0):
                if(tem_genome_gff_chr_gene.iloc[0]['strand']=='-'):
                    
                    #CDS of gene
                    start =  tem_genome_gff_chr_gene.iloc[0]['start']
                    end =  tem_genome_gff_chr_gene.iloc[0]['end']
                    tem_genome_gff_chr_cds = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                    tem_genome_gff_chr_cds = tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['end']<=end].copy()
                    tem_genome_gff_chr_cds =  tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['Genome_attr']=='CDS'].copy()
                    max_end_cds = tem_genome_gff_chr_cds['end'].max()
                    tem_genome_gff_chr_cds = tem_genome_gff_chr_cds[tem_genome_gff_chr_cds['end'] == max_end_cds].copy()
                    
                    # tRNA and rRNA without CDS, replace it 
                    if(len(tem_genome_gff_chr_cds)==0):
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['end']<=end].copy()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['Genome_attr']=='rRNA'].copy()
                        max_end_rRNA = tem_genome_gff_chr_rRNA['end'].max()
                        tem_genome_gff_chr_rRNA = tem_genome_gff_chr_rRNA[tem_genome_gff_chr_rRNA['end']==max_end_rRNA].copy()
                        
                        if(len(tem_genome_gff_chr_rRNA)==0):
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr[tem_genome_gff_chr['start']>=start].copy()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['end']<=end].copy()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['Genome_attr']=='tRNA'].copy()
                            max_end_tRNA = tem_genome_gff_chr_tRNA['end'].max()
                            tem_genome_gff_chr_tRNA = tem_genome_gff_chr_tRNA[tem_genome_gff_chr_tRNA['end']==max_end_tRNA].copy()
                            
                            # no tRNA,rRNA and CDS, report error.
                            if(len(tem_genome_gff_chr_tRNA)==0):
                                print("error! There is no CDS & tRNA & rRNA in upstream!!!")
                                print(tem_genome_gff_chr_gene)
                                print("\n\n")
                            else:
                                CHIP_seq_output.iloc[i,7] = tem_genome_gff_chr_tRNA.iloc[0]['ID']
                                CHIP_seq_output.iloc[i,8] = binding_site- tem_genome_gff_chr_tRNA.iloc[0]['end']
                                                            
                        else:
                            CHIP_seq_output.iloc[i,7] = tem_genome_gff_chr_rRNA.iloc[0]['ID']
                            CHIP_seq_output.iloc[i,8] = binding_site- tem_genome_gff_chr_rRNA.iloc[0]['end']
                    
                    
                    
                    else:
                        #write information to CHIP_seq_output
                        CHIP_seq_output.iloc[i,7]= tem_genome_gff_chr_cds.iloc[0]['ID']
                        CHIP_seq_output.iloc[i,8]=binding_site- tem_genome_gff_chr_cds.iloc[0]['end']
                
                else:
                    CHIP_seq_output.iloc[i,7]=None
                    CHIP_seq_output.iloc[i,8]=None
            else:
                print("Nearst upstream attribute is not gene!",end = " ")
                print("chr=%s, peak summit=%s"%(CHIP_seq_output.iloc[i][1], binding_site))
                print()
                    
                    
                    

    if(i%100==0):
        print("i={},precentage={:.2f}%".format(i,i/len(CHIP_seq_output)*100))
print('end')
        

i=0,precentage=0.00%
Nearst downstream attribute is not gene! chr=ChrA_C_glabrata_CBS138, peak summit=488681

Nearst downstream attribute is not gene! chr=ChrG_C_glabrata_CBS138, peak summit=990110

i=100,precentage=39.84%
Nearst downstream attribute is not gene! chr=ChrA_C_glabrata_CBS138, peak summit=484545

i=200,precentage=79.68%
end


In [11]:
CHIP_seq_output

,type,chr,start,end,abs_summit,ctrl_gene_down,down_distance,ctrl_gene_up,up_distance,ingenebody,Tss_distance,code_strand
0,common,ChrA_C_glabrata_CBS138,57463,57663,57563,ID=CAGL0A00517g-P;Parent=CAGL0A00517g-T;orf_cl...,1267,ID=CAGL0A00473g-P;Parent=CAGL0A00473g-T;orf_cl...,4066,ID=CAGL0A00495g;Name=CAGL0A00495g;Gene=PMA1;No...,194,-
1,common,ChrA_C_glabrata_CBS138,130041,130241,130141,ID=CAGL0A01284g-P;Parent=CAGL0A01284g-T;orf_cl...,1221,None,None,ID=CAGL0A01243g;Name=CAGL0A01243g;Gene=GIT1;No...,1070,-
2,common,ChrA_C_glabrata_CBS138,160382,160582,160482,ID=CAGL0A01628g-P;Parent=CAGL0A01628g-T;orf_cl...,845,ID=CAGL0A01606g-P;Parent=CAGL0A01606g-T;orf_cl...,568,None,None,None
3,common,ChrA_C_glabrata_CBS138,186433,186633,186533,None,None,ID=CAGL0A01826g-P;Parent=CAGL0A01826g-T;orf_cl...,754,None,None,None
4,common,ChrA_C_glabrata_CBS138,194065,194265,194165,ID=CAGL0A01892g-P;Parent=CAGL0A01892g-T;orf_cl...,395,ID=CAGL0A01848g-P;Parent=CAGL0A01848g-T;orf_cl...,3461,ID=CAGL0A01870g;Name=CAGL0A01870g;Note=Has%20d...,556,-
...,...,...,...,...,...,...,...,...,...,...,...,...
246,TH0_specific,ChrL_C_glabrata_CBS138,1094565,1094765,1094665,None,None,ID=CAGL0L10142g-P;Parent=CAGL0L10142g-T;orf_cl...,625,None,None,None
247,TH0_specific,ChrL_C_glabrata_CBS138,1350210,1350410,1350310,ID=CAGL0L12540g-P;Parent=CAGL0L12540g-T;orf_cl...,1919,ID=CAGL0L12496g-P;Parent=CAGL0L12496g-T;orf_cl...,1257,ID=CAGL0L12518g;Name=CAGL0L12518g;Note=Protein...,1665,-
248,TH0_specific,ChrL_C_glabrata_CBS138,1425087,1425287,1425187,ID=CAGL0L13299g-P;Parent=CAGL0L13299g-T;orf_cl...,1507,ID=CAGL0L13288g-P;Parent=CAGL0L13288g-T;orf_cl...,2466,None,None,None
249,TH0_specific,ChrM_C_glabrata_CBS138,708771,708971,708871,None,None,ID=CAGL0M06941g-P;Parent=CAGL0M06941g-T;orf_cl...,515,ID=CAGL0M06963g;Name=CAGL0M06963g;Note=Ortholo...,332,+


In [14]:
CHIP_seq_output['suggested_gene']=None
CHIP_seq_output['suggested_gene_distance']=None

In [15]:
for i in range(len(CHIP_seq_output)):
    distance=[]
    if(CHIP_seq_output.iloc[i]['down_distance']!=None):
        distance.append(CHIP_seq_output.iloc[i]['down_distance'])
    if(CHIP_seq_output.iloc[i]['up_distance']!=None):
        distance.append(CHIP_seq_output.iloc[i]['up_distance'])
    
    if distance:
        min_dis=min(distance)
#         if(CHIP_seq_output.iloc[i]['Tss_distance']==min_dis):
#             CHIP_seq_output.iloc[i,22]=CHIP_seq_output.iloc[i]['ingenebody']

        if(CHIP_seq_output.iloc[i]['down_distance']==min_dis):
            CHIP_seq_output.iloc[i,12]=CHIP_seq_output.iloc[i]['ctrl_gene_down']
            CHIP_seq_output.iloc[i,13]=CHIP_seq_output.iloc[i]['down_distance']

        if(CHIP_seq_output.iloc[i]['up_distance']==min_dis):
            CHIP_seq_output.iloc[i,12]=CHIP_seq_output.iloc[i]['ctrl_gene_up']
            CHIP_seq_output.iloc[i,13]=CHIP_seq_output.iloc[i]['up_distance']


            
            

In [16]:
CHIP_seq_output

,type,chr,start,end,abs_summit,ctrl_gene_down,down_distance,ctrl_gene_up,up_distance,ingenebody,Tss_distance,code_strand,suggested_gene,suggested_gene_distance
0,common,ChrA_C_glabrata_CBS138,57463,57663,57563,ID=CAGL0A00517g-P;Parent=CAGL0A00517g-T;orf_cl...,1267,ID=CAGL0A00473g-P;Parent=CAGL0A00473g-T;orf_cl...,4066,ID=CAGL0A00495g;Name=CAGL0A00495g;Gene=PMA1;No...,194,-,ID=CAGL0A00517g-P;Parent=CAGL0A00517g-T;orf_cl...,1267
1,common,ChrA_C_glabrata_CBS138,130041,130241,130141,ID=CAGL0A01284g-P;Parent=CAGL0A01284g-T;orf_cl...,1221,None,None,ID=CAGL0A01243g;Name=CAGL0A01243g;Gene=GIT1;No...,1070,-,ID=CAGL0A01284g-P;Parent=CAGL0A01284g-T;orf_cl...,1221
2,common,ChrA_C_glabrata_CBS138,160382,160582,160482,ID=CAGL0A01628g-P;Parent=CAGL0A01628g-T;orf_cl...,845,ID=CAGL0A01606g-P;Parent=CAGL0A01606g-T;orf_cl...,568,None,None,None,ID=CAGL0A01606g-P;Parent=CAGL0A01606g-T;orf_cl...,568
3,common,ChrA_C_glabrata_CBS138,186433,186633,186533,None,None,ID=CAGL0A01826g-P;Parent=CAGL0A01826g-T;orf_cl...,754,None,None,None,ID=CAGL0A01826g-P;Parent=CAGL0A01826g-T;orf_cl...,754
4,common,ChrA_C_glabrata_CBS138,194065,194265,194165,ID=CAGL0A01892g-P;Parent=CAGL0A01892g-T;orf_cl...,395,ID=CAGL0A01848g-P;Parent=CAGL0A01848g-T;orf_cl...,3461,ID=CAGL0A01870g;Name=CAGL0A01870g;Note=Has%20d...,556,-,ID=CAGL0A01892g-P;Parent=CAGL0A01892g-T;orf_cl...,395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,TH0_specific,ChrL_C_glabrata_CBS138,1094565,1094765,1094665,None,None,ID=CAGL0L10142g-P;Parent=CAGL0L10142g-T;orf_cl...,625,None,None,None,ID=CAGL0L10142g-P;Parent=CAGL0L10142g-T;orf_cl...,625
247,TH0_specific,ChrL_C_glabrata_CBS138,1350210,1350410,1350310,ID=CAGL0L12540g-P;Parent=CAGL0L12540g-T;orf_cl...,1919,ID=CAGL0L12496g-P;Parent=CAGL0L12496g-T;orf_cl...,1257,ID=CAGL0L12518g;Name=CAGL0L12518g;Note=Protein...,1665,-,ID=CAGL0L12496g-P;Parent=CAGL0L12496g-T;orf_cl...,1257
248,TH0_specific,ChrL_C_glabrata_CBS138,1425087,1425287,1425187,ID=CAGL0L13299g-P;Parent=CAGL0L13299g-T;orf_cl...,1507,ID=CAGL0L13288g-P;Parent=CAGL0L13288g-T;orf_cl...,2466,None,None,None,ID=CAGL0L13299g-P;Parent=CAGL0L13299g-T;orf_cl...,1507
249,TH0_specific,ChrM_C_glabrata_CBS138,708771,708971,708871,None,None,ID=CAGL0M06941g-P;Parent=CAGL0M06941g-T;orf_cl...,515,ID=CAGL0M06963g;Name=CAGL0M06963g;Note=Ortholo...,332,+,ID=CAGL0M06941g-P;Parent=CAGL0M06941g-T;orf_cl...,515


In [17]:
CHIP_seq_output['suggested_gene']=CHIP_seq_output['suggested_gene'].str.split(';',expand=True)[0]
CHIP_seq_output['suggested_gene']=CHIP_seq_output['suggested_gene'].str.split('=',expand=True)[1]

In [18]:
CHIP_seq_output.to_excel(output_path,index=False)